In [ ]:
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", None)  # this will be helpful when visualizing retriever outputs

### 0. Load dataset

In [ ]:
import datasets
from tqdm.notebook import tqdm
from langchain.docstore.document import Document as LangchainDocument

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Save each data as a list of document
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)
]

### 1. Retriever - embeddings
<p>The retriever acts like an internal search engine: given the user query, it returns a few relevant snippets from your knowledge base. These snippets will then be fed to the Reader Model to help it generate its answer.</p>
How many snippets should we retrieve? This parameter will be named <b>top_k</b>. <br>
How long should these snippets be? This is called the <b>chunk size</b>.

<i>"In this notebook, we use Langchain library since it offers a huge variety of options for vector databases and allows us to keep document metadata throughout the processing."</i>

#### 1.1 Split the documents into chunks
- In this part, we split the documents from our knowledge base into smaller chunks which will be the snippets on which the reader LLM will base its answer.
- The goal is to prepare a collection of semantically relevant snippets. So their size should be adapted to precise ideas: too small will truncate ideas, too large will dilute them.

<i>Many options exist for text splitting: splitting on words, on sentence boundaries, recursive chunking that processes documents in a tree-like way to preserve structure information…</i>

- <b>Recursive chunking</b> breaks down the text into smaller parts step by step using a given list of separators sorted from the most important to the least important separator. <br>
For instance with the list of separators ["\n\n", "\n", ".", ""]:
    - The method will first break down the document wherever there is a double line break "\n\n".
    - Resulting documents will be split again on simple line breaks "\n", then on sentence ends ".".
    - And finally, if some chunks are still too big, they will be split whenever they overflow the maximum size.  

We also keep in mind that before we split the documents into chuncks we have to embed the documents and it could be a problem that the chunk lengths are not aligned with our limit maximum sequence length of the embedding model. </br>
<b>So we should make sure that our chunck sizes are below this limit, because any longer chunk will be trucated before processing, thus losing relevancy.</b>

In [ ]:
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = 'thenlper/gte-small'

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter: 512.
print(f"Model's maximum sequence length: {SentenceTransformer(EMBED_MODEL_NAME).max_seq_length}")

Now that we already know the maximum length of the embedding model we can create a RecursiveCharacterTextSplitter to count length in number of tokens instead of number of characters and then we can choose specific chunck size. To do that we indicate in the splitter the tokenizer we will use.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

# This list is taken from LangChain's MarkdownTextSplitter class.
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_documents(chunk_size: int, knowledge_base: List[LangchainDocument], tokenizer_name: Optional[str] = EMBED_MODEL_NAME, ) -> List[LangchainDocument]:
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name), # we use the same tokenizer model as the embedding model
        chunk_size=chunk_size,  # the maximum number of characters in a chunk: we selected this value arbitrarily
        chunk_overlap=int(chunk_size/10),  # the number of characters to overlap between chunks
        add_start_index=True,  # If `True`, includes chunk's start index in metadata
        strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBED_MODEL_NAME,
)

#### 1.2 Building the vector database
We want to compute the embeddings for all the chunks of our knowledge base.</br> We first convert the chunks tokenized into embeddings.

##### <b>How does retrieval work?</b> </br>
Once the chunks are all embedded, we store them into a vector database. When the user types in a query, it gets embedded by the same model previously used, and a similarity search returns the closest documents from the vector database.

The technical challenge is thus, given a query vector, to quickly find the nearest neighbours of this vector in the vector database. <br>
To do this, we need to choose two things: a <b>distance</b>, and a <b>search algorithm</b> to find the nearest neighbors quickly within a database of thousands of records.

##### <b>Nearest Neighbor search algorithm</b>
There are plentiful choices for the nearest neighbor search algorithm: we go with Facebook’s <b>FAISS</b>, since FAISS is performant enough for most use cases, and it is well known thus widely implemented.

##### Distances
We will use <b>Cosine similarity</b>. <br> It computes similarity between two vectors as the cosinus of their relative angle: it allows us to compare vector directions are regardless of their magnitude. Using it requires to normalize all vectors, to rescale them into unit norm.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

We want to find the k document that have the closest meaning, thus we pick the k closest vectors.<br>
In the LangChain vector database implementation, this search operation is performed by the method vector_database.similarity_search(query).

In [ ]:
# embed a user query in the same space
user_query = "How to create a pipeline object?"
query_vector = embedding_model.embed_query(user_query)

print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print("\n==================================Top document==================================")
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)

In [ ]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

In [ ]:
import torch

torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)